# Agent

에이전트의 핵심 아이디어는 언어 모델을 사용하여 수행할 일련의 동작을 선택하는 것입니다. 체인에서는 일련의 동작이 코드에 하드코딩되어 있습니다. 그러나 에이전트에서는 언어 모델이 어떤 동작을 취할지 및 어떤 순서로 취할지를 결정하는 추론 엔진으로 사용됩니다.

다음은 다음 단계를 결정하는데 책임을 지고 있는 체인입니다. 이는 언어 모델과 프롬프트에 의해 제어됩니다. 이 체인의 입력은 다음과 같습니다:

1. 도구(Tools): 사용 가능한 도구에 대한 설명
2. 사용자 입력(User Input): 고수준 목표
3. 중간 단계(Intermediate Steps): 사용자 입력을 달성하기 위해 이전에 순서대로 실행된 (동작, 도구 출력) 쌍

출력은 다음 동작 또는 사용자에게 보낼 최종 응답(AgentActions 또는 AgentFinish)입니다. 동작은 도구와 해당 도구에 대한 입력을 지정합니다.

다양한 에이전트는 추론에 대한 다양한 프롬프팅 스타일, 입력 인코딩 방식 및 출력 파싱 방식을 가지고 있습니다. 내장된 에이전트의 전체 목록은 에이전트 유형을 참조하세요. 아래의 시작하기 섹션에서 사용자 정의 에이전트를 어떻게 만드는지 보여주는 것도 가능합니다.

In [2]:
# Agent가 단순한 LLM보다 좋은 경우를 확인해보자 
# 아래의 단순한 연산의 결과를 보면 LLM이 틀린 답을 출력해주는 것을 볼 수 있음. 
# LLM은 text를 완성하는 것입니다. 그냥 다음 token에 대해 예측하는 것이죠.

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

AIMessage(content='The total cost is $4,353.38.')

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(a,b): # 사용할 function을 만들기 
    return a+b 

agent = initialize_agent(
    llm=llm, 
    verbose=True, # agent가 어떤 작업을 수행하는지 출력해줌.
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # 어떤 종류의 agent로 initialize할 것인지 지정해줌.
    tools=[ # tools는 여러개를 지정할 수 있음. 
    StructuredTool.from_function(
        func=plus,
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers. till finish."
    ),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I will use the Sum Calculator tool to find the total cost of all these amounts.
Thought: The Sum Calculator tool will help us quickly add up all the given costs.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```

Observation: 1280.26
Thought:The sum of $355.39 and $924.87 is $1280.26. Next, I will continue adding the remaining amounts using the Sum Calculator tool.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:The sum of $355.39, $924.87, and $721.2 is $2001.46. Next, I will continue adding the remaining amounts using the Sum Calculator tool. 

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:The sum of $355.39, $924.87, $721.2, and $1940.29 is $3941.75. Next, I will continue adding the remaining amounts using the 

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

### How do agents work 

agent가 끝날 때까지 반복되는 loop가 있음. (Langchain 문서에서 AgentExecutor를 보면 pseudocode가 있음.)
agent에 input을 줄 때 runtime은 LLM으로부터 get_action을 받아오려 시도함. LLM에게 어떤 액션을 할지 고르라고 할 것입니다. 


### Zero-Shot ReAct Agent 

agent에는 다양한 유형이 있습니다. 이번에 알아볼 Agent는 Zero-Shot ReAct입니다.

계속해서 `OpenAI Functions`에 집중해서 살펴볼 것입니다. 이건 LLM으로부터 출력 값을 강요하는 것보다 훨씬 나은 함수를 호출하는 방식을 사용합니다.

그러면 우리가 걱정하는 Agent가 제대로 작동하지 않을 것에 대한 위험을 줄일 수 있습니다.

`Zero-shot ReAct`는 가장 범용적인 목적의 Agent입니다. 만약 `OpenAI Functions`을 사용하지 않는다면(GPT-3 or GPT-4 turbo) `Zero-shot ReAct`가 함수 호출을 지원하지 않는 다른 모델들과 같이 사용하게 될 친구입니다. 


`Zero-shot ReAct`과 `Structured Input ReAct`가 다른 점은 Structured Input은 여러 입력을 가질 수 있지만 Zero-shot은 그렇지 않다는 것입니다. 

Reasoning과 Acting은 추론과 연기를 의미하는데요, 아래의 논문은 해당 내용과 관련된 내용입니다. 여기에 Agent를 어떻게 활용하고 agent가 잘 동작하기 위해 어떤 지시를 줘야하는지 써있습니다. 
(관련 논문 : https://arxiv.org/pdf/2210.03629.pdf)

---

중요한 부분은 원한다면 이 agent를 LCEL을 사용해서 자신의 것으로 활용할 수 있다는 것입니다. 

In [8]:
from langchain.tools import Tool  # zero shot react에서는 StructuredTool 대신 Tool을 사용해야함.

llm = ChatOpenAI(temperature=0.1)

def plus(input): # 사용할 function을 만들기 
    a, b = input.split(",") 
    return float(a) + float(b)

agent = initialize_agent(
    llm=llm, 
    verbose=True, # agent가 어떤 작업을 수행하는지 출력해줌.
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # zero shot react description을 사용
    handle_parsing_errors = True, # LLM이 우리가 원하는 출력 형식으로 응답하지 않는다. output parse에 파싱 에러가 발생하면 이 코드가 고치려고함
    tools=[ # tools는 여러개를 지정할 수 있음. 
    Tool.from_function(
        func=plus,
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma. \nExample: 1,2",
    ),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I should use the Sum Calculator to add up all the given numbers.
Action: Sum Calculator
Action Input: 355.39,924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 1280.26,721.2
Observation: 2001.46
Thought:I should keep adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 2001.46,1940.29
Observation: 3941.75
Thought:I need to keep adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 3941.75,573.63
Observation: 4515.38
Thought:I need to keep adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4515.38,65.72
Observation: 4581.1
Thought:I should continue adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4581.1,35.00
Observation: 4616.1
Thought:I need to keep adding the remaining numbers to the sum.
Action: Sum Calculator
Action Input: 4616.1,552.00
Observation: 5168.1
Th

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5244.26'}

### OpenAI Functions Agent 

OPENAI 함수를 사용하기 위해선 다른 방법을 통해 Tool을 설정해야한다. 

In [10]:
from langchain.tools import BaseTool  # zero shot react에서는 StructuredTool 대신 Tool을 사용해야함.
from pydantic import BaseModel, Field
from typing import Type 

llm = ChatOpenAI(temperature=0.1)

class CalculatorToolArgsSchema(BaseModel):
    a: int = Field(description="The first number")
    b: int = Field(description="The second number") 

class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arugments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self,a,b):
        return a+b

agent = initialize_agent(
    llm=llm, 
    verbose=True, # agent가 어떤 작업을 수행하는지 출력해줌.
    agent=AgentType.OPENAI_FUNCTIONS, # openai functions을 사용
    handle_parsing_errors = True, # LLM이 우리가 원하는 출력 형식으로 응답하지 않는다. output parse에 파싱 에러가 발생하면 이 코드가 고치려고함
    tools=[
        CalculatorTool()
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 35539, 'b': 92487}`


128026
Invoking: `CalculatorTool` with `{'a': 128026, 'b': 72120}`


200146
Invoking: `CalculatorTool` with `{'a': 200146, 'b': 194029}`


394175
Invoking: `CalculatorTool` with `{'a': 394175, 'b': 57363}`


451538
Invoking: `CalculatorTool` with `{'a': 451538, 'b': 6572}`


458110
Invoking: `CalculatorTool` with `{'a': 458110, 'b': 3500}`


461610
Invoking: `CalculatorTool` with `{'a': 461610, 'b': 55200}`


516810
Invoking: `CalculatorTool` with `{'a': 516810, 'b': 7616}`


524426
Invoking: `CalculatorTool` with `{'a': 524426, 'b': 2912}`


527338The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}